# Analysis of differential expression across conditions

In [2]:
library("DESeq2")
setwd("~/Documents/Proyecto_rodopsina/")
load("Data/DokdoniaTPMCounts.RData")

In [3]:
cnames <- colnames(data$counts)
conditions = unlist(
    lapply(cnames, function(s){return(strsplit(s, ".sam")[[1]])})
)

light_conditions <- conditions[1:16]
dark_conditions <- conditions[17:length(conditions)]

light <- vector()
temp <- vector()
files <- colnames(data$counts)
for (i in 1:length(files)) {
    light <- c(light, substring(files[i], 1, 1))
    temp <- c(temp, substring(files[i], 3, 4))
}

meta <- data.frame(row.names = cnames,
                  "Temperature" = temp,
                  "Light" = light)
head(meta, 5)

,Temperature,Light
,<chr>,<chr>
D_10_R1.sam,10,D
D_10_R2.sam,10,D
D_10_R3.sam,10,D
D_18_R1.sam,18,D
D_18_R2.sam,18,D


In [4]:
dds <- DESeqDataSetFromMatrix(countData = data$counts, 
                              colData = meta, design = ~ Temperature)  

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”


In [5]:
dds_lrt <- DESeq(dds, test="LRT", reduced = ~ 1) # Likelyhood Ratio Test (multiple conditions)

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [15]:
res <- results(dds_lrt, alpha = 1e-5)
res

log2 fold change (MLE): Temperature 34 vs 10 
LRT p-value: '~ Temperature' vs '~ 1' 
DataFrame with 2886 rows and 6 columns
              baseMean log2FoldChange     lfcSE      stat       pvalue
             <numeric>      <numeric> <numeric> <numeric>    <numeric>
MED134_07389  28698.32       0.941887 0.0757369  179.5902  1.08133e-38
MED134_07384   3909.36       1.130010 0.0596776  399.6192  2.67690e-86
MED134_07379   6942.78       0.903971 0.0447610  469.1036 2.36562e-101
MED134_07374   2984.57       0.364880 0.0643950   51.0036  4.88341e-11
MED134_07369   3262.90       0.204430 0.0856888   17.1243  6.66336e-04
...                ...            ...       ...       ...          ...
MED134_14782   1833.07      -0.405973 0.0907462   31.9619  5.33119e-07
MED134_14787   4857.90      -0.332865 0.0398016   99.4119  2.07938e-21
MED134_14792   4789.24      -0.937420 0.0681734  202.0328  1.53433e-43
MED134_14797   1070.22       0.234741 0.0494182  154.9895  2.20969e-33
MED134_14802   6407.24  

In [16]:
summary(res)


out of 2886 with nonzero total read count
adjusted p-value < 1e-05
LFC > 0 (up)       : 1053, 36%
LFC < 0 (down)     : 1101, 38%
outliers [1]       : 0, 0%
low counts [2]     : 0, 0%
(mean count < 15)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [8]:
?results

results {DESeq2},R Documentation
object,"a DESeqDataSet, on which one of the following functions has already been called: DESeq, nbinomWaldTest, or nbinomLRT"
contrast,"this argument specifies what comparison to extract from the object to build a results table. one of either: a character vector with exactly three elements: the name of a factor in the design formula, the name of the numerator level for the fold change, and the name of the denominator level for the fold change (simplest case) a list of 2 character vectors: the names of the fold changes for the numerator, and the names of the fold changes for the denominator. these names should be elements of resultsNames(object). if the list is length 1, a second element is added which is the empty character vector, character(). (more general case, can be to combine interaction terms and main effects) a numeric contrast vector with one element for each element in resultsNames(object) (most general case) If specified, the name argument is ignored."
name,"the name of the individual effect (coefficient) for building a results table. Use this argument rather than contrast for continuous variables, individual effects or for individual interaction terms. The value provided to name must be an element of resultsNames(object)."
lfcThreshold,"a non-negative value which specifies a log2 fold change threshold. The default value is 0, corresponding to a test that the log2 fold changes are equal to zero. The user can specify the alternative hypothesis using the altHypothesis argument, which defaults to testing for log2 fold changes greater in absolute value than a given threshold. If lfcThreshold is specified, the results are for Wald tests, and LRT p-values will be overwritten."
altHypothesis,"character which specifies the alternative hypothesis, i.e. those values of log2 fold change which the user is interested in finding. The complement of this set of values is the null hypothesis which will be tested. If the log2 fold change specified by name or by contrast is written as beta , then the possible values for altHypothesis represent the following alternate hypotheses: greaterAbs: |beta| > lfcThreshold , and p-values are two-tailed lessAbs: |beta| < lfcThreshold , p-values are the maximum of the upper and lower tests. The Wald statistic given is positive, an SE-scaled distance from the closest boundary greater: beta > lfcThreshold less: beta < -lfcThreshold"
listValues,"only used if a list is provided to contrast: a numeric of length two: the log2 fold changes in the list are multiplied by these values. the first number should be positive and the second negative. by default this is c(1,-1)"
cooksCutoff,"theshold on Cook's distance, such that if one or more samples for a row have a distance higher, the p-value for the row is set to NA. The default cutoff is the .99 quantile of the F(p, m-p) distribution, where p is the number of coefficients being fitted and m is the number of samples. Set to Inf or FALSE to disable the resetting of p-values to NA. Note: this test excludes the Cook's distance of samples belonging to experimental groups with only 2 samples."
independentFiltering,"logical, whether independent filtering should be applied automatically"
alpha,"the significance cutoff used for optimizing the independent filtering (by default 0.1). If the adjusted p-value cutoff (FDR) will be a value other than 0.1, alpha should be set to that value."
filter,the vector of filter statistics over which the independent filtering will be optimized. By default the mean of normalized counts is used.
